In [6]:
import sys
sys.prefix

'/home/osx/anaconda3/envs/cs224n'

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [8]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 37.7 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
# # Install required libraries
# !pip install datasets
# !pip install transformers==4.28.0
# !pip install accelerate
# !pip install jiwer
# !pip install huggingface_hub

In [5]:
# Import libraries
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer
from huggingface_hub import notebook_login

/home/osx/anaconda3/envs/cs224n/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/osx/anaconda3/envs/cs224n/lib/python3.11/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
2023-11-14 04:40:12.897810: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2

In [5]:
notebook_login()

map columns for noisy data

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file into a DataFrame
df = pd.read_csv('data/sentences_noise_data_sample.csv')

# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create a DataFrame for the training set
train_subset = pd.DataFrame({
    'Sentence': train_df['Sentence'],
    'noisy_sentence': train_df.apply(lambda row: row[row['Best_Alternative']], axis=1)
})

# Create a DataFrame for the testing set
test_subset = pd.DataFrame({
    'Sentence': test_df['Sentence'],
    'noisy_sentence': test_df.apply(lambda row: row[row['Best_Alternative']], axis=1)
})

# Save the training and testing sets to new CSV files
train_subset.to_csv('data/train_sentences_noise_data.csv', index=False)
test_subset.to_csv('data/test_sentences_noise_data.csv', index=False)


convert to json

In [18]:
import pandas as pd

def convert_csv_to_json(csv_path, json_output_path):

    df = pd.read_csv(csv_path)

    json_data = df.to_json(orient='records')


    with open(json_output_path, 'w') as json_file:
        json_file.write(json_data)

csv_file_path = 'data/train_sentences_noise_data.csv'
json_output_file_path = 'data/train_output.json'
convert_csv_to_json(csv_file_path, json_output_file_path)

In [19]:
csv_file_path = 'data/test_sentences_noise_data.csv'
json_output_file_path = 'data/test_output.json'
convert_csv_to_json(csv_file_path, json_output_file_path)

In [21]:
# Load the dataset from the JSON files
dataset = load_dataset('json', data_files={'train': 'data/train_output.json',
                                           'test': 'data/test_output.json'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [22]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'noisy_sentence'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['Sentence', 'noisy_sentence'],
        num_rows: 2000
    })
})


In [23]:
print(dataset['train'][:5])
print(dataset['test'][:5])

{'Sentence': ["It's not easy to win.", "I don't want you to tell Tom about what happened here.", "She's in the National Theatre Company.", 'You should distinguish between right and wrong.', 'I knew that Tom was a coward.'], 'noisy_sentence': ['itznot  e azyt ow in', 'idom twamdy oud o telltom  ap outwhat  happenedhe re', 's hesi nthenatiomar d heatrec onpa my', 'distinku is hbedwee mligh dandwro mg  youwer eb robab', 'ikm ow dhaddom i s outraged ikn ew dhad tomw asac o']}
{'Sentence': ['Tom sat in the dentist’s chair.', "He didn't say what kind of bike he has.", "I'd be happy to come in.", 'Good afternoon! How are you?', "I told you I didn't want this."], 'noisy_sentence': ['dond oogou this false t eeth', 'cy cre heditn t s ayw h adki mdov bikehe haz', 'i t be happ ytocom eim', 'hello a r eyous dilldhele', 'idolty ou  ititn   dwamddhis']}


In [24]:
# Split the dataset into train and validation sets
train_dataset, val_dataset = train_test_split(dataset['train'], test_size=0.1, random_state=42)

In [26]:
from datasets import Dataset

train_data = Dataset.from_dict(train_dataset)  # Convert the train data to a dataset
val_data = Dataset.from_dict(val_dataset)      # Convert the validation data to a dataset

In [27]:
print(type(train_data))
print(type(val_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [28]:
# Check column names in train dataset
print(train_data.column_names)

# Check column names in validation dataset
print(val_data.column_names)

['Sentence', 'noisy_sentence']
['Sentence', 'noisy_sentence']


In [30]:
print(train_data['Sentence'][:5])
print(train_data['noisy_sentence'][:5])
#print(val_data[:5])

["Do you know Tom's full name?", "Today I went to the dentist's.", "You'll ask Tom for permission to do that, won't you?", 'Morocco got into debt.', "The reason that Tom isn't here is because he's sick in bed."]
['do youk now to msvur  lnam e', 'yiwe mt tothete ndists  inee dtoothbas de', 'i n su red omd hatw  ouldtod h ati vyouazgethim to', 'mor o c cogodintod ebt', 'tom izi nbe dbecaus ehezn odfeerin g werl thereaz omthatt om iz mdherei zpe  cau z ehessi ']


In [31]:
# Define the source and target language columns
source_lang = 'noisy_sentence'
target_lang = 'Sentence'

In [32]:
print(source_lang)

noisy_sentence


In [33]:
# Define the max_length for padding and truncation
max_length = 512

In [34]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs


In [35]:
# Select a random data point from the train dataset
sample_data = train_data[0]

# Call the preprocess function on the sample data
processed_data = preprocess_function(sample_data)

# Inspect the output
print(processed_data)

{'input_ids': tensor([[    0,  2362, 26351,  1215, 19530,  4086,    35,   385,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1021,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1437,  1437,     2],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1423,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1021,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1717,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,   449,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1437,  1437,     2],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,   295,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1021,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,   885,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1437,  1437,     2],
        [    0,  2362, 263

In [36]:
#train_data = preprocess_function(train_data)
#val_data = preprocess_function(val_data)

# Apply preprocess_function to train_data and val_data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [37]:
# Access a few samples from train_data
for i in range(5):
    sample_input_ids = train_data['input_ids'][i]
    sample_attention_mask = train_data['attention_mask'][i]
    sample_labels = train_data['labels'][i]

    print(f"Sample {i+1}:")
    print("Input IDs:", sample_input_ids)
    print("Attention Mask:", sample_attention_mask)
    print("Labels:", sample_labels)
    print()

Sample 1:
Input IDs: [0, 2362, 26351, 1215, 19530, 4086, 35, 109, 47, 330, 122, 7, 43601, 705, 710, 1437, 784, 8697, 364, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Labels: [0, 8275, 47, 216, 1560, 18, 455, 766, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [38]:
# Prepare DataLoader for training and validation
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

In [39]:
# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch

In [40]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="spell_correction_tatoeba",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    push_to_hub=True,
)

In [41]:
# verify passing the correct inputs to the trainer
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)
print("Tokenizer:", tokenizer)
print("Training Arguments:", training_args)

Train Dataset: Dataset({
    features: ['Sentence', 'noisy_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7200
})
Validation Dataset: Dataset({
    features: ['Sentence', 'noisy_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 800
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=Tru

In [42]:
# model is initialized with the BARTForConditionalGeneration class and moved to the GPU if available.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

KeyboardInterrupt: 